In [11]:
import requests
API_KEY ="nPQal8j3vHTC1BvE1LkhVw50ibcLtwwz0p2ucnnv"
url=f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"
response = requests.get(url)
data = response.json() # Dic Format

In [12]:
data.keys()

dict_keys(['links', 'element_count', 'near_earth_objects'])

In [15]:
import streamlit as st

In [17]:
data['links']['next'] # to get next link of pagination

'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-08&end_date=2024-01-15&detailed=false&api_key=nPQal8j3vHTC1BvE1LkhVw50ibcLtwwz0p2ucnnv'

In [4]:
data['near_earth_objects'] #having list  inside the dictionary
data['near_earth_objects'].keys() #keys from dic 

# to get data from 'near_earth_objects'
details = data['near_earth_objects']['2024-01-02'][0] #1st asteroid info
details = data['near_earth_objects']['2024-01-02']

In [ ]:
pip --version


In [19]:
import datetime
from datetime import datetime

In [21]:
#check for 10k records (pagination)using while loop
asteroids_data = []
target = 10000
url=f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"
response = requests.get(url)
data = response.json()
details = data['near_earth_objects']

while len(asteroids_data) < target:
  for date,info in details.items():
      for i in info:
        asteroids_data.append(dict(
        id = int(i['id']),
        neo_reference_id= int(i['neo_reference_id']) ,              
        name = format(i['name']),
        mag= float(i['absolute_magnitude_h']),
        estimated_diam_max_km= float(i['estimated_diameter']['kilometers']['estimated_diameter_min']),
        estimated_diam_min_km = float(i['estimated_diameter']['kilometers']['estimated_diameter_max']),
        is_potentially_hazardous_asteroid=bool(i['is_potentially_hazardous_asteroid']),
        close_approach_date =  datetime.strptime(i['close_approach_data'][0]['close_approach_date'],'%Y-%m-%d').date(),
        relative_velocity_kmph=float(i['close_approach_data'][0]['relative_velocity']['kilometers_per_second']),
        astronomical= float(i['close_approach_data'][0]['miss_distance']['astronomical']),
        miss_distance_km= float(i['close_approach_data'][0]['miss_distance']['kilometers']),
        miss_distance_lunar=float(i['close_approach_data'][0]['miss_distance']['lunar']),
        orbiting_body=i['close_approach_data'][0]['orbiting_body']))
        if len(asteroids_data) >= target:
            break
      if len(asteroids_data) >= target:
          break
  url = data['links'].get(data['links']['next'])



print (url) #to navigate to next set of record, if no records/no dates error should not be thrown , using get() will return none if no values 

None


In [23]:
len(asteroids_data)

10000

In [25]:

asteroids_data[201]


{'id': 3780672,
 'neo_reference_id': 3780672,
 'name': '(2017 PM26)',
 'mag': 21.28,
 'estimated_diam_max_km': 0.1474195145,
 'estimated_diam_min_km': 0.3296400556,
 'is_potentially_hazardous_asteroid': False,
 'close_approach_date': datetime.date(2024, 1, 6),
 'relative_velocity_kmph': 18.612928453,
 'astronomical': 0.2157438397,
 'miss_distance_km': 32274818.88474144,
 'miss_distance_lunar': 83.9243536433,
 'orbiting_body': 'Earth'}

In [27]:
pip install mysqlx-connector-python

Note: you may need to restart the kernel to use updated packages.


In [28]:
!pip install mysql-connector-python
#import packages
import mysql.connector as db
import pandas as pd



In [31]:
type(asteroids_data[0])

dict

In [33]:

connection =db.connect(
    host ="localhost",
    user="saran",
    password="Sure@1206",
    database ="nasa_db")

cursor =connection.cursor()



In [27]:
cursor.execute ("""create table asteroids (
                id int,
name varchar(50),
absolute_magnitude_h float,
estimated_diam_max_km float,
estimated_diam_min_km float,
is_potentially_hazardous_asteroid varchar(6));""")



NameError: name 'cursor' is not defined

In [28]:
insert_query="""insert into asteroids 
(id,name,absolute_magnitude_h,
estimated_diam_max_km,estimated_diam_min_km,
is_potentially_hazardous_asteroid)values
(%s,%s,%s,%s,%s,%s);"""


In [ ]:
asteroid=[]

In [30]:
for i in asteroids_data:
    # print(i['id'],i['name'],i['mag'])
     values = (i['id'],i['name'],i['mag'],i['estimated_diam_max_km'],i['estimated_diam_min_km'],i['is_potentially_hazardous_asteroid'])
     cursor.execute(insert_query,values)
connection.commit()

In [31]:
cursor.execute ("""create table close_approach(
               neo_reference_id int,
close_approach_date date,
relative_velocity_kmph float,
astronomical float,
miss_distance_km float,
miss_distance_lunar float,
orbiting_body varchar(50));""")


In [34]:
insert_query="""insert into close_approach 
(neo_reference_id,close_approach_date,relative_velocity_kmph,
astronomical,miss_distance_km,
miss_distance_lunar,orbiting_body)values
(%s,%s,%s,%s,%s,%s,%s);"""


In [45]:
for i in asteroids_data:
    # print(i['id'],i['name'],i['mag'])
     values = (i['neo_reference_id'],i['close_approach_date'],i['relative_velocity_kmph'],i['astronomical'],
               i['miss_distance_km'],i['miss_distance_lunar'],i['orbiting_body'])
     cursor.execute(insert_query,values)
connection.commit()

NameError: name 'insert_query' is not defined

#Count how many times each asteroid has approached Earth

In [35]:
from tabulate import tabulate

query1 = "select id, count(distinct(id)) from nasa_db.asteroids group by 1;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = ["Asteroids_id", "count"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))


+----------------+---------+
|   Asteroids_id |   count |
+================+=========+
|        2168318 |       1 |
+----------------+---------+
|        2199003 |       1 |
+----------------+---------+
|        2304640 |       1 |
+----------------+---------+
|        2415949 |       1 |
+----------------+---------+
|        2434188 |       1 |
+----------------+---------+
|        2450293 |       1 |
+----------------+---------+
|        2585310 |       1 |
+----------------+---------+
|        2591179 |       1 |
+----------------+---------+
|        2613286 |       1 |
+----------------+---------+
|        2668499 |       1 |
+----------------+---------+
|        2669051 |       1 |
+----------------+---------+
|        2676480 |       1 |
+----------------+---------+
|        3102728 |       1 |
+----------------+---------+
|        3102756 |       1 |
+----------------+---------+
|        3154493 |       1 |
+----------------+---------+
|        3160747 |       1 |
+-------------

#Average velocity of each asteroid over multiple approaches

In [41]:
query1 = "SELECT neo_reference_id, AVG(relative_velocity_kmph) AS Avg_Veloicty FROM nasa_db.close_approach group by neo_reference_id;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = ["Asteroids_id", "Avg_Velocity"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))


+----------------+----------------+
|   Asteroids_id |   Avg_Velocity |
+================+================+
|        2415949 |      15.8905   |
+----------------+----------------+
|        3160747 |      10.7192   |
+----------------+----------------+
|        3309828 |      15.6703   |
+----------------+----------------+
|        3457842 |       6.08087  |
+----------------+----------------+
|        3553062 |       8.74138  |
+----------------+----------------+
|        3591616 |       5.807    |
+----------------+----------------+
|        3608936 |      11.0228   |
+----------------+----------------+
|        3795154 |      18.9936   |
+----------------+----------------+
|        3842680 |      20.8331   |
+----------------+----------------+
|        2613286 |      15.6703   |
+----------------+----------------+
|       54338714 |       5.30003  |
+----------------+----------------+
|       54415081 |      11.3259   |
+----------------+----------------+
|       54417653 |      12.2

#List top 10 fastest asteroids #relative_velocity_kmph - Speed of the asteroid during approach (in km/h)

In [49]:
query1 = "SELECT distinct(relative_velocity_kmph),neo_reference_id AS Asteroid_ID FROM nasa_db.close_approach ORDER BY relative_velocity_kmph DESC LIMIT 10;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Max_Velocity","ID"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+----------------+----------+
|   Max_Velocity |       ID |
+================+==========+
|        37.8521 |  3645001 |
+----------------+----------+
|        33.8373 |  2434188 |
+----------------+----------+
|        26.4969 |  3655366 |
+----------------+----------+
|        25.3932 |  3724393 |
+----------------+----------+
|        24.9092 | 54151575 |
+----------------+----------+
|        23.7179 |  2450293 |
+----------------+----------+
|        23.0326 |  2304640 |
+----------------+----------+
|        21.6522 |  3789479 |
+----------------+----------+
|        21.5014 | 54419807 |
+----------------+----------+
|        21.464  | 54162245 |
+----------------+----------+


#Find potentially hazardous asteroids that have approached Earth more than 3 times

In [89]:
asteroids_data[0]


{'id': 2415949,
 'neo_reference_id': 2415949,
 'name': '415949 (2001 XY10)',
 'mag': 19.37,
 'estimated_diam_max_km': 0.3552670883,
 'estimated_diam_min_km': 0.7944013596,
 'is_potentially_hazardous_asteroid': False,
 'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
 'relative_velocity_kmph': 15.8905264223,
 'astronomical': 0.3372535274,
 'miss_distance_km': 50452409.349026635,
 'miss_distance_lunar': 131.1916221586,
 'orbiting_body': 'Earth'}

In [37]:
query1 = "SELECT id,is_potentially_hazardous_asteroid, count(is_potentially_hazardous_asteroid) FROM nasa_db.asteroids where is_potentially_hazardous_asteroid='1'group by 1,2 having(count(is_potentially_hazardous_asteroid)>3);"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Asteroids","Is_Hazardous","Frequency_of_Hazardous_Asteroids"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+-------------+----------------+------------------------------------+
|   Asteroids |   Is_Hazardous |   Frequency_of_Hazardous_Asteroids |
+=============+================+====================================+
|     3309828 |              1 |                                 77 |
+-------------+----------------+------------------------------------+
|     3608936 |              1 |                                 77 |
+-------------+----------------+------------------------------------+
|     2613286 |              1 |                                 77 |
+-------------+----------------+------------------------------------+
|     2669051 |              1 |                                 77 |
+-------------+----------------+------------------------------------+
|     2450293 |              1 |                                 77 |
+-------------+----------------+------------------------------------+
|     3440771 |              1 |                                 77 |
+-------------+-----

#Find the month with the most asteroid approaches

In [41]:
query1 = "SELECT DATE_FORMAT(close_approach_date, '%Y-%m') AS ym, COUNT(*) AS approach_count FROM nasa_db.close_approach GROUP BY ym ORDER BY approach_count DESC;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Asteroids","Max Asteroids every month"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+-------------+-----------------------------+
| Asteroids   |   Max Asteroids every month |
+=============+=============================+
| 2024-01     |                       10000 |
+-------------+-----------------------------+


In [99]:
asteroids_data[0]

{'id': 2415949,
 'neo_reference_id': 2415949,
 'name': '415949 (2001 XY10)',
 'mag': 19.37,
 'estimated_diam_max_km': 0.3552670883,
 'estimated_diam_min_km': 0.7944013596,
 'is_potentially_hazardous_asteroid': False,
 'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
 'relative_velocity_kmph': 15.8905264223,
 'astronomical': 0.3372535274,
 'miss_distance_km': 50452409.349026635,
 'miss_distance_lunar': 131.1916221586,
 'orbiting_body': 'Earth'}

In [103]:
query1 = "SELECT neo_reference_id, max(relative_velocity_kmph) AS Max_speed FROM nasa_db.close_approach group by neo_reference_id limit 1;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Asteroid_Id","Max_speed"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+---------------+-------------+
|   Asteroid_Id |   Max_speed |
+===============+=============+
|       2415949 |     15.8905 |
+---------------+-------------+


In [ ]:
query1 = "Select max(estimated_diam_max_km) as max_diam,id from nasa_db.asteroids group by id ORDER BY max_diam DESC ;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Asteroid_Id","Max_speed"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

In [ ]:
query1 = "Select distinct ca.neo_reference_id,a.name,ca.close_approach_date,ca.miss_distance_km from nasa_db.close_approach as ca inner join nasa_db.asteroids as a on ca.neo_reference_id = a.id;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Asteroid_Id","Name","close_approach_date","miss_distance_km"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

In [59]:
query1 = "Select distinct ca.neo_reference_id,a.name,ca.close_approach_date,ca.miss_distance_km ,ca.relative_velocity_kmph,relative_velocity_kmph=ca.relative_velocity_kmph * 1000 as 'velocity' from nasa_db.close_approach as ca inner join nasa_db.asteroids as a on ca.neo_reference_id = a.id where 'velocity' >5000;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Asteroid_Id","Name","close_approach_date","miss_distance_km","velocity"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+---------------+--------+-----------------------+--------------------+------------+
| Asteroid_Id   | Name   | close_approach_date   | miss_distance_km   | velocity   |
+===============+========+=======================+====================+============+
+---------------+--------+-----------------------+--------------------+------------+


In [46]:
query1 = "Select month(close_approach_date) as month,count(neo_reference_id)  from nasa_db.close_approach group by 1 order by month asc;"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Month","count_Approaches"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+---------+--------------------+
|   Month |   count_Approaches |
+=========+====================+
|       1 |              10000 |
+---------+--------------------+


In [43]:
query1 = "Select distinct id,absolute_magnitude_h  from nasa_db.asteroids where absolute_magnitude_h = (Select min(absolute_magnitude_h) from nasa_db.asteroids);"
cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Month","count_Approaches"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+---------+--------------------+
|   Month |   count_Approaches |
+=========+====================+
| 2199003 |              18.16 |
+---------+--------------------+


In [45]:
query1 = "Select id,count(id) as cnt_Hazardous_Asteroid from  nasa_db.asteroids where is_potentially_hazardous_asteroid =1 group by 1;"

cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Id","cnt_Hazardous_Asteroid"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+----------+--------------------------+
|       Id |   cnt_Hazardous_Asteroid |
+==========+==========================+
|  3309828 |                       77 |
+----------+--------------------------+
|  3608936 |                       77 |
+----------+--------------------------+
|  2613286 |                       77 |
+----------+--------------------------+
|  2669051 |                       77 |
+----------+--------------------------+
|  2450293 |                       77 |
+----------+--------------------------+
|  3440771 |                       77 |
+----------+--------------------------+
|  3794988 |                       77 |
+----------+--------------------------+
|  2585310 |                       77 |
+----------+--------------------------+
|  2168318 |                       77 |
+----------+--------------------------+
|  3102728 |                       77 |
+----------+--------------------------+
|  2199003 |                       77 |
+----------+--------------------------+


In [ ]:
query1 = "Select id,count(id) as cnt_non_Hazardous_Asteroid from  nasa_db.asteroids where is_potentially_hazardous_asteroid =0 group by 1;"

cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Id","cnt_non_Hazardous_Asteroid"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

In [49]:
query1 = "SELECT  a.name AS asteroid_name, ca.close_approach_date, ca.miss_distance_km FROM  close_approach ca JOIN  asteroids a ON a.id = ca.neo_reference_id WHERE ca.orbiting_body = 'Earth' AND ca.miss_distance_lunar<1 ORDER BY ca.miss_distance_km asc;"

cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Id","Dis_asteroid_closer_than_moon","close_approach_date","Distance_in_Km"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+-----------+---------------------------------+-----------------------+
| Id        | Dis_asteroid_closer_than_moon   |   close_approach_date |
+===========+=================================+=======================+
| (2024 AD) | 2024-01-04                      |                242676 |
+-----------+---------------------------------+-----------------------+
| (2024 AD) | 2024-01-04                      |                242676 |
+-----------+---------------------------------+-----------------------+
| (2024 AD) | 2024-01-04                      |                242676 |
+-----------+---------------------------------+-----------------------+
| (2024 AD) | 2024-01-04                      |                242676 |
+-----------+---------------------------------+-----------------------+
| (2024 AD) | 2024-01-04                      |                242676 |
+-----------+---------------------------------+-----------------------+
| (2024 AD) | 2024-01-04                      |                2

#15. Find asteroids that came within 0.05 AU(astronomical distance)

In [47]:
query1 = "Select neo_reference_id,astronomical from nasa_db.close_approach where astronomical<=0.05;"

cursor.execute(query1)
result1 = cursor.fetchall()

# Define table headers
headers = [ "Id","astronomical"]

# Print the result as a table
print(tabulate(result1, headers=headers, tablefmt="grid"))

+----------+----------------+
|       Id |   astronomical |
+==========+================+
|  3842680 |     0.0260973  |
+----------+----------------+
| 54417653 |     0.0115641  |
+----------+----------------+
| 54418497 |     0.00408142 |
+----------+----------------+
| 54419485 |     0.0438287  |
+----------+----------------+
| 54418438 |     0.0166083  |
+----------+----------------+
| 54418650 |     0.0271321  |
+----------+----------------+
| 54419698 |     0.00970741 |
+----------+----------------+
| 54421384 |     0.0491416  |
+----------+----------------+
| 54418649 |     0.00162219 |
+----------+----------------+
| 54419602 |     0.00497407 |
+----------+----------------+
| 54419705 |     0.0348077  |
+----------+----------------+
| 54419812 |     0.0179964  |
+----------+----------------+
| 54419817 |     0.0179149  |
+----------+----------------+
| 54418498 |     0.0154699  |
+----------+----------------+
| 54418648 |     0.0259174  |
+----------+----------------+
| 54419186

In [51]:
%%writefile nasa_demo3.py

import streamlit as st
import pandas as pd
from datetime import datetime
import sqlite3
import pymysql
import base64

import matplotlib.pyplot as plt

conn = pymysql.connect(host="localhost",user="root",password="Sure@1206",database="nasa_db")
cursor=conn.cursor()
st.set_page_config(layout="wide")
st.markdown("<h1 style='text-align: center;color: #1108BBE'> Insight -Nasa Asteroid </h1>", unsafe_allow_html=True)
#st.image(r"C:/Users/Hashvetha/OneDrive/Desktop/ast2.jpg")
with open(r"C:/Users/Hashvetha/Downloads/nasaim.webp", "rb") as image_file:
 
    img_base64 = base64.b64encode(image_file.read()).decode('utf-8')
st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url("data:image/jpg;base64,{img_base64}");
        background-size: cover;
        background-repeat: no-repeat;
        background-attachment: fixed;
    }}
    </style>
    """,
    unsafe_allow_html=True
)
st.divider()

with st.sidebar:
 page = st.sidebar.radio("Go to", ["About Nasa Project", "Filter Criterias", "SQL-Asteroids Data"])

# -------------------------------- PAGE 1: Introduction --------------------------------
if page == "About Nasa Project":
    st.title(" NEO Data Analysis")
    st.subheader(" A Streamlit App for Analysing Asteroids & to Explore Astronomical data.")
    st.write("""
    This project analyzes Asteroids,astronomical data month wise using an MYSQL database.
    It provides visualizations for Asteroid with highest Brightness, Asteroid Speed, count of hazardous vs non-hazardous asteroids
    and asteroid with the most close approaches.
    **Features:**
    - View and filter asteroids data by month, close approaches
    - Generate dynamic visualizations.
    - Run predefined SQL queries to explore insights.
    **Database Used:** `Nasa_DB`
    """)

# -------------------------------- PAGE 2:  Data Visualization --------------------------------

elif page=="SQL-Asteroids Data":
        
 #with st.sidebar:

               options = st.selectbox("Queries",[
                      "1. No.of Times each asteroid has approached Earth(count)",
                      "2. Average velocity of each asteroid over multiple approaches",
                      "3. Top 10 fastest asteroids",
                      "4. Potentially hazardous asteroids that have approached Earth more than 3 times",
                      "5. Month with the most asteroid approaches",
                      "6. Asteroid with the fastest ever approach speed",
                      "7. Asteroids by maximum estimated diameter (descending)",
                      "8. Asteroid whose closest approach is getting nearer over time",
                      "9. Asteroids name, date & miss_distance of Asteroids closest approach to Earth.",
                      "10. Names of asteroids that approached Earth with velocity > 50,000 km/h",
                      "11. Count of approaches happened per month",
                      "12. Asteroid with the highest brightness",
                      "13. Number of hazardous asteroids",
                      "14. Number of non-hazardous asteroids",
                      "15. Asteroids,close_approach_date and distance that passed closer than the Moon",
                      "16. Asteroids that came within 0.05 AU(astronomical distance)"
                      ],placeholder='Select an option...',index=None)
               if options == "1. No.of Times each asteroid has approached Earth(count)":
                        cursor.execute('select id, count(distinct(id)) from asteroids group by 1')
                        result =  cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data) 

               elif options == "2. Average velocity of each asteroid over multiple approaches":
                        cursor.execute('select neo_reference_id, AVG(relative_velocity_kmph) AS Avg_Veloicty FROM close_approach group by neo_reference_id')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data) 

               elif options == "3. Top 10 fastest asteroids":
                        cursor.execute('select distinct(relative_velocity_kmph),neo_reference_id AS Asteroid_ID FROM close_approach ORDER BY relative_velocity_kmph DESC LIMIT 10')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data) 
               elif options == "4. Potentially hazardous asteroids that have approached Earth more than 3 times":
                        cursor.execute("SELECT a.name,ca.neo_reference_id, COUNT(*) as close_approach_count from nasa_db.close_approach as ca inner join nasa_db.asteroids as a on ca.neo_reference_id = a.id WHERE a.is_potentially_hazardous_asteroid = 0 AND orbiting_body = 'Earth'GROUP BY a.name,ca.neo_reference_id HAVING COUNT(*) > 3 ORDER BY close_approach_count DESC")
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data) 


               elif options == "5. Month with the most asteroid approaches":
                        cursor.execute('select neo_reference_id, AVG(relative_velocity_kmph) AS Avg_Veloicty FROM close_approach group by neo_reference_id')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data) 

               elif options == "6. Asteroid with the fastest ever approach speed":
                        cursor.execute('select neo_reference_id, AVG(relative_velocity_kmph) AS Avg_Veloicty FROM close_approach group by neo_reference_id')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data) 
                
               elif options == "7. Asteroids by maximum estimated diameter (descending)":
                        cursor.execute('select neo_reference_id, AVG(relative_velocity_kmph) AS Avg_Veloicty FROM close_approach group by neo_reference_id')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)
                
               elif options == "8. Asteroid whose closest approach is getting nearer over time":
                        cursor.execute('select a.name,COUNT(c.neo_reference_id) AS approach_count FROM asteroids a JOIN close_approach c ON a.id = c.neo_reference_id GROUP BY a.name ORDER BY approach_count DESC')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "9. Asteroids name, date & miss_distance of Asteroids closest approach to Earth.":
                        cursor.execute("select a.name,ca.close_approach_date,ca.miss_distance_km FROM close_approach ca JOIN  asteroids a ON a.id = ca.neo_reference_id WHERE ca.orbiting_body = 'Earth'")
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "10. Names of asteroids that approached Earth with velocity > 50,000 km/h":
                        cursor.execute("select DISTINCT a.name,ca.relative_velocity_kmph FROM close_approach ca JOIN asteroids a ON a.id = ca.neo_reference_id WHERE ca.orbiting_body = 'Earth'AND ca.relative_velocity_kmph*10000")
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "11. Count of approaches happened per month":
                        cursor.execute("SELECT DATE_FORMAT(close_approach_date, '%Y-%d') as ym, count(*) FROM close_approach GROUP BY ym ORDER BY ym")
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "12. Asteroid with the highest brightness":
                        cursor.execute('SELECT name,  absolute_magnitude_h FROM asteroids ORDER BY  absolute_magnitude_h ASC LIMIT x`1')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "13. Number of hazardous asteroids":
                        cursor.execute('select id,count(id) as cnt_Hazardous_Asteroid from  nasa_db.asteroids where is_potentially_hazardous_asteroid =1 group by 1')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "14. Number of non-hazardous asteroids":
                        cursor.execute('select id,count(id) as cnt_Non_Hazardous_Asteroid from  nasa_db.asteroids where is_potentially_hazardous_asteroid =0 group by 1')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "15. Asteroids,close_approach_date and distance that passed closer than the Moon":
                        cursor.execute("SELECT  a.name AS asteroid_name, ca.close_approach_date, ca.miss_distance_km FROM  close_approach ca JOIN  asteroids a ON a.id = ca.neo_reference_id WHERE ca.orbiting_body = 'Earth' AND ca.miss_distance_lunar<1 ORDER BY ca.miss_distance_km asc")
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
                
               elif options == "16. Asteroids that came within 0.05 AU(astronomical distance)":
                        cursor.execute('Select neo_reference_id,astronomical from close_approach where astronomical<=0.05')
                        result = cursor.fetchall()
                        columns = [desc[0] for desc in cursor.description]
                        data = pd.DataFrame(result,columns=columns)
                        st.dataframe(data)  
# -------------------------------- PAGE 3: Filter --------------------------------




elif page =="Filter Criterias":
 c1,a,c2,b,c3=st.columns([0.4,0.4,0.4,0.4,0.4])
 with c1:
    mag_min=st.slider("Min Magnitude",19.00,19.38,(19.00,19.38 ))

#Diameter
    diam_min=st.slider("Min Est Diam in KM",0.70,0.800,value=(0.70,0.800))
    diam_max=st.slider("Max Est Diam in KM",0.30,0.800,value=(0.30,0.800))

 with c2:
    velocity=st.slider("Rel_velocity",15,20,value=(15,20))
    astro=st.slider("Astro Unit",5.16453e-05,0.4999515747)
    hazardous=st.selectbox("only show pot hazardous",options =[0,1],index=0)

 with c3:
    start_date=st.date_input("start date",datetime(2024,1,1))
    end_date=st.date_input("End date",datetime(2025,1,13))

 if st.button("Apply Filters"):
        query = f"""
            SELECT *
            FROM nasa_db.asteroids a
            JOIN nasa_db.close_approach c ON a.id = c.neo_reference_id
            WHERE a.absolute_magnitude_h  BETWEEN {mag_min[0]} AND {mag_min[1]}
              AND a.estimated_diam_min_km  BETWEEN {diam_min[0]} AND {diam_min[1]}
              AND a.estimated_diam_max_km  BETWEEN {diam_max[0]} AND {diam_max[1]}
              AND c.relative_velocity_kmph BETWEEN {velocity[0]} AND {velocity[1]}
              AND c.astronomical >= {astro}
              AND a.is_potentially_hazardous_asteroid = {hazardous}
              AND DATE(c.close_approach_date) BETWEEN DATE('{start_date}') AND DATE('{end_date}')
            LIMIT 10000;
        """

        cursor.execute(query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]

        if result:
            df = pd.DataFrame(result, columns=columns)
            st.dataframe(df)
        else:
            st.warning("No asteroids found for the given filter criteria.")

    




Overwriting nasa_demo3.py


In [ ]:
!streamlit run nasa_demo3.py